In [1]:
! git clone https://github.com/huggingface/transformers.git > /dev/null

Cloning into 'transformers'...
remote: Enumerating objects: 73567, done.
remote: Total 73567 (delta 0), reused 0 (delta 0), pack-reused 73567
Receiving objects: 100% (73567/73567), 56.38 MiB | 32.51 MiB/s, done.
Resolving deltas: 100% (52336/52336), done.


In [5]:
! pip install -e transformers/ 

Obtaining file:///content/transformers
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 901kB 7.0MB/s 
     |████████████████████████████████| 3.3MB 15.7MB/s 
  Running setup.py develop for transformers


In [6]:
! pip install datasets > /dev/null

In [7]:
import pandas as pd

In [8]:
df = pd.read_csv('/content/drive/MyDrive/other/generate_bz_lyrics/df_all.csv')
print(df.shape)
df.head()

(347, 6)


,URL,SongName,Artist,Lyricist,Composer,lyrics
0,https://www.uta-net.com/song/54892/,I WANNA DANCE Wicked Beat Style,B'z,Kohshi Inaba,Takahiro Matsumoto,Crush! Crush! Taxi on a busy neon streetGive m...
1,https://www.uta-net.com/song/7938/,あいかわらずなボクら,B'z,稲葉浩志,松本孝弘,今まで好きなこともしたし たまに我慢もしてきた「あいつはダメだ」 なんてキミ勝手に決めないで...
2,https://www.uta-net.com/song/17979/,愛と憎しみのハジマリ,B'z,KOSHI INABA,TAK MATSUMOTO,誰もが愛する者 守ろうとしてる頃 憎しみがスキをうかがう大破綻の雰囲気が世界中を取り巻いて朝...
3,https://www.uta-net.com/song/25415/,愛のバクダン,B'z,稲葉浩志,松本孝弘,今日も一日 不完全な世界きみが泣かなくても 誰か泣く人がこしらえた 悲しみのからくりなんとか...
4,https://www.uta-net.com/song/26049/,愛のprisoner,B'z,KOSHI INABA,TAK MATSUMOTO,"I hate you, hate you こらえきれずに夜の闇に吠える 涙流さないあなたのど..."


In [9]:
def remove_full_width_and_wow(text):
    removed = text.replace('\u3000', '')
    removed = removed.replace('\n', '')
    removed = removed.replace('wow', '')
    removed = removed.replace('NA', '')
    return removed
lyrics = df['lyrics']
lyrics = [remove_full_width_and_wow(lyric) for lyric in lyrics]

In [10]:
with open('/content/drive/MyDrive/other/generate_bz_lyrics/preprocessed.txt', mode='w') as f:
    f.write('\n\n'.join(lyrics))

重みの容量が膨大なため、driveに書き出すことを非推奨

In [16]:
!python transformers/examples/pytorch/language-modeling/run_clm.py  \
    --output_dir='finetune_gpt2' \
    --model_type=gpt2 \
    --model_name_or_path=gpt2 \
    --do_train \
    --train_file='/content/drive/MyDrive/other/generate_bz_lyrics/preprocessed.txt' \
    --do_eval \
    --validation_file='/content/drive/MyDrive/other/generate_bz_lyrics/preprocessed.txt' \
    --per_device_train_batch_size=3 \
    --per_device_eval_batch_size=3 \
    --num_train_epochs=100 \
    --save_total_limit=3 \
    --overwrite_output_dir

2021-05-28 04:03:44.617506: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
05/28/2021 04:03:46 - WARNING - __main__ -   Process rank: -1, device: cuda:0, n_gpu: 1distributed training: False, 16-bits training: False
05/28/2021 04:03:46 - INFO - __main__ -   Training/evaluation parameters TrainingArguments(output_dir=finetune_gpt2, overwrite_output_dir=True, do_train=True, do_eval=True, do_predict=False, evaluation_strategy=IntervalStrategy.NO, prediction_loss_only=False, per_device_train_batch_size=3, per_device_eval_batch_size=3, gradient_accumulation_steps=1, eval_accumulation_steps=None, learning_rate=5e-05, weight_decay=0.0, adam_beta1=0.9, adam_beta2=0.999, adam_epsilon=1e-08, max_grad_norm=1.0, num_train_epochs=100.0, max_steps=-1, lr_scheduler_type=SchedulerType.LINEAR, warmup_ratio=0.0, warmup_steps=0, logging_dir=runs/May28_04-03-46_a39fa63a1155, logging_strategy=IntervalStrategy.STEPS, logging_first_step=Fa

In [18]:
!python transformers/examples/pytorch/text-generation/run_generation.py \
    --model_type=gpt2 \
    --model_name_or_path='finetune_gpt2' \
    --prompt "夏の日" \
    --length=600 \
    --num_return_sequences=20

2021-05-28 06:50:17.549640: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
05/28/2021 06:50:18 - WARNING - __main__ -   device: cuda, n_gpu: 1, 16-bits training: False
05/28/2021 06:50:23 - INFO - __main__ -   Namespace(device=device(type='cuda'), fp16=False, k=0, length=600, model_name_or_path='finetune_gpt2', model_type='gpt2', n_gpu=1, no_cuda=False, num_return_sequences=20, p=0.9, padding_text='', prefix='', prompt='夏の日', repetition_penalty=1.0, seed=42, stop_token=None, temperature=1.0, xlm_language='')
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
=== GENERATED SEQUENCE 1 ===
夏の日にとこへとしない楽しい僕は長い夜も許してくれるような気がしたらどうするつもりが話せない泣かないでいるのに溢れてるまで忘れるまで踊らむ以上時間どこにだってYour Invitation君を開ければ失うまで忘れないまで踊らむ以上逢うまくいいで夢を見ただろう泣かないで思わずかないで笑ってるまで夜を開ければいいまで忘れないまで泣かないで忘れないまで忘れないまで忘れないまで踊らむ以上逢うまくいいで夢を見ただろう思わずかないで笑ってるまで夜を開ければいいまで忘れないまで忘れないまで泣かないで何もわからないで愛しあってるまで忘れないまで泣かないで何もわからないで愛しあってるまで忘れないまで泣かないで夢を見ただろう泣かないで思わず

In [19]:
!python transformers/examples/pytorch/text-generation/run_generation.py \
    --model_type=gpt2 \
    --model_name_or_path='finetune_gpt2' \
    --prompt "波" \
    --length=600 \
    --num_return_sequences=20

2021-05-28 06:51:01.124734: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
05/28/2021 06:51:02 - WARNING - __main__ -   device: cuda, n_gpu: 1, 16-bits training: False
05/28/2021 06:51:06 - INFO - __main__ -   Namespace(device=device(type='cuda'), fp16=False, k=0, length=600, model_name_or_path='finetune_gpt2', model_type='gpt2', n_gpu=1, no_cuda=False, num_return_sequences=20, p=0.9, padding_text='', prefix='', prompt='波', repetition_penalty=1.0, seed=42, stop_token=None, temperature=1.0, xlm_language='')
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
=== GENERATED SEQUENCE 1 ===
波を見つけるまた一人狂いまくろう人よどこにゃない自分よどこかに溶けてしまう夢はいつもどこかで続くのだろう言葉ひとつ足りないよNow is the time狂いまくろう人よどこにゃないどこかに溶けてしまう夢はいつもどこかで続くのだろう手のぬくもり返しげなくなり抱きしめたいとこまで行ってみようことを気づいているように何もないとただ確かに思ってたどりつくのだろうまっさい君のことだけでいいそれまでもこの世界に思ってたいそれはまだ見ぬ笑うほど愛しい人よNow is the time行きまくろう人よどこにゃないどこかに溶けてしまう夢はいつもどこかで続くのだろう行ってみようことを気づいているように何もないとただ確かに思ってたどりつくのだろ

In [20]:
!python transformers/examples/pytorch/text-generation/run_generation.py \
    --model_type=gpt2 \
    --model_name_or_path='finetune_gpt2' \
    --prompt "まだ暑い" \
    --length=600 \
    --num_return_sequences=20

2021-05-28 06:55:07.363594: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
05/28/2021 06:55:08 - WARNING - __main__ -   device: cuda, n_gpu: 1, 16-bits training: False
05/28/2021 06:55:13 - INFO - __main__ -   Namespace(device=device(type='cuda'), fp16=False, k=0, length=600, model_name_or_path='finetune_gpt2', model_type='gpt2', n_gpu=1, no_cuda=False, num_return_sequences=20, p=0.9, padding_text='', prefix='', prompt='まだ暑い', repetition_penalty=1.0, seed=42, stop_token=None, temperature=1.0, xlm_language='')
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
=== GENERATED SEQUENCE 1 ===
まだ暑いたら何をあげて楽しい日々の流れ生きてる歩き出す少々の口をつけようぜ泣いたことだけどそんなに世界が次の誰かを品定めるだろう信じていたけど正直なやつがたまらなけりゃ朝が来たんだと思わせるんだ気づかないものを守りたいと感じたら逃げていただ欲望さらしい日々の流れ落ちてゆく世界が次の誰かを品定めるだろう気づかないものを守りたいと感じたら逃げていただ欲望さらしい日々の流れ生きてる歩き出すそんなに泣いたことだけどそんなに世界が次の誰かを品定めるだろう歩き出すそんなに泣いたことだけどそんなに世界が次の誰かを品定めるだろう息を飲み干してくれる声を聞かせて止まりを歩いてゆくと言い聞かせて逢いただ欲望さらしい日々の流れ生きてる歩き出